In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from typing import List
from pydantic import BaseModel
import os
import json
import csv

from llama_parse import LlamaParse
from llama_index.core.schema import Document
from llama_index.llms.openai import OpenAI
import nest_asyncio

nest_asyncio.apply()

In [4]:
## Parse the Form Filing File
def parse_file(file_path: str) -> List[Document]:
    llama_parse = LlamaParse(
        api_key=os.environ['LLAMA_CLOUD_API_KEY'],
        result_type='markdown',
        target_pages="0"
    )
    result = llama_parse.load_data(
        file_path,
    )
    return result

In [5]:
documents = parse_file('data/Report_format_2.xlsx')

Started parsing the file under job_id 79c04060-47e8-44f8-9c6e-bd414c4be17d


In [6]:
print(f'Doc length: {type(documents)}')
print(len(documents))

Doc length: <class 'list'>
1


In [7]:
print(f'Doc length: {len(documents)}')
print(documents[0].text)

Doc length: 1
|Items to be improved                                                                         |Description|Improvement Parameters|                  |                   |                 |                        |                      |                   |            |        |
|---------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------------------|------------------|-------------------|-----------------|------------------------|----------------------|-------------------|------------|--------|
|                                                                                             | |Improvement direction |Person responsible|Expected start date|Actual start date|Expected completion date|Actual c

In [8]:
text = documents[0].text

In [9]:
## Structured Extraction
prompt = f"""
You are an AI assistant specializing in Industrial Engineering problem solving. You've been given an Excel spreadsheet containing items to be improved. \ 
and improvement parameters. Your task is to extract and structure this information in a clear, organized format.

The Excel sheet contains the following:
1. Items to be improved (rows)
2. Description (columns)
3. Improvement parameters and dates (columns)

Input Excel data:
{text}

Please present the extracted and structured information in a clear, easy-to-read format.
"""


In [10]:
class ReportParameters(BaseModel):
    """Data model for IE problem solving analysis."""
    ItemsToBeImproved: List[str]
    Description: List[str]
    ImprovementParameters: List[str]

In [11]:
llm = OpenAI(model='gpt-4o-mini')

In [12]:
from llama_index.core.llms import ChatMessage
input_msg = ChatMessage.from_str('What is your name?')
output = llm.chat([input_msg])
output.message.content

'I’m called ChatGPT. How can I assist you today?'

In [14]:
from llama_index.core.llms import ChatMessage
sllm = llm.as_structured_llm(output_cls=ReportParameters)
input_msg = ChatMessage.from_str(prompt)

In [15]:
output = sllm.chat([input_msg])
output_obj = output.raw
output_obj

ReportParameters(ItemsToBeImproved=['The saturation of the lower glass point Xiaoli Pill is 65%', 'Bottom glass electrophoresis tank+UVFixed baking operation saturation79.2%', 'Xiaoliwan wax+Paste conductive foam+Lower glass glue frame dispensing operation saturation75%', 'The upper and lower glass bonding saturation is 67.5%', 'Five-in-one front&back saturation53.2%', 'CellstickFPCsaturation61.9%', 'FPCFoam resistance test saturation40.8%', 'The saturation of amplified pills and sports cars is 73.3%', 'pointUVglue,UVsolidify&Check saturation33.3%', 'Electrophoresis tank noteBufferliquid saturation79.2%', 'BufferLiquid injection port sealing and solidification+Water leakage test saturation60.3%', 'Silicone oil saturation 50.7%', 'Silicone oil port sealing + dispensing + curing saturation 58%', 'Set Conn & lower cover + paste QR CODE saturation 41%', 'Lock the lid + label the saturation 81.7%', 'Glass module test saturation 100%', 'Visual inspection of UV glue + pull tape is the bottlen

In [16]:
#print out the items based on the data model
print(len(output_obj.ItemsToBeImproved))
print(len(output_obj.Description))
print(len(output_obj.ImprovementParameters))
output_obj.ImprovementParameters

22
22
10


['',
 'Improvement direction',
 'Person responsible',
 'Expected start date',
 'Actual start date',
 'Expected completion date',
 'Actual completion date',
 'Improve immediately',
 'Confirmation',
 'appendix']

In [17]:
ItemsToBeImproved = output_obj.ItemsToBeImproved
Description = output_obj.Description
ImprovementParameters = output_obj.ImprovementParameters[1:]

In [18]:
ImprovementParameters

['Improvement direction',
 'Person responsible',
 'Expected start date',
 'Actual start date',
 'Expected completion date',
 'Actual completion date',
 'Improve immediately',
 'Confirmation',
 'appendix']

In [19]:
from llama_index.indices.managed.llama_cloud import LlamaCloudIndex
# pip install llama-index-indices-managed-llama-cloud
index = LlamaCloudIndex(
  name="objective-wildfowl-2024-11-24", 
  project_name="Default",
  organization_id="2033a7fc-187e-48e4-a172-5079c4ee2bbf",
  api_key=os.environ['LLAMA_CLOUD_API_KEY']
)

In [20]:
query_engine = index.as_query_engine(
    dense_similarity_top_k=10,
    sparse_similarity_top_k=10,
    alpha=0.5,
    enable_reranking=True,
    rerank_top_n=5,
)

In [22]:
# sllm.chat(['What is your name?'])
Description = ['Low job saturation(higher than95%)',
 'Low job saturation(higher than99%)',
 'Low job saturation(lower than90%)']
Description

['Low job saturation(higher than95%)',
 'Low job saturation(higher than99%)',
 'Low job saturation(lower than90%)']

In [23]:
#Generate query and test if it generating them correctly
from tqdm import tqdm
items_to_be_improved = {}

for index, item in enumerate(ItemsToBeImproved[:3]):
    text_info = Description[index]
    job_saturation = (
    f"Analyze the following text and determine whether the job saturation value mentioned is lower than 95%.\n\n"
    f"Text: \"{text_info}\"\n\n"
    f"If a job saturation value is explicitly mentioned, check if it is lower than 95%. If so, respond with 'YES'. "
    f"If it is 95% or higher, respond with 'NO'. If no job saturation value is mentioned, respond with 'NO INFORMATION'."
)
    input_msg = ChatMessage.from_str(job_saturation)
    output = llm.chat([input_msg])
    answer = output.message.content
    
    print(index,',',text_info,',',answer)
    
    # for (index, parameter) in enumerate(ImprovementParameters):
    #     des = Description[index]
    #     job_saturation = f"Is the job saturation value lower than 95% for:'{des}'? Your response should be boolean. Either YES or NO?"
    #     # query = f"What is the '{parameter}' for '{item}'? If you don't know the answer then say 'NA'"
    #     answer = str(query_engine.query(job_saturation))
    #     # items_to_be_improved[item][parameter] = answer
    #     print(des,',',answer)
    #     index +=1

0 , Low job saturation(higher than95%) , NO
1 , Low job saturation(higher than99%) , NO
2 , Low job saturation(lower than90%) , YES


In [24]:
text_info = 'My country name is Bangladsh. I do not know the value of job saturation at this point of time!'
# text_info = 'low job saturation (lower than 99%)'
# text_info = 'Simultaneous working time of man and machine = manual time (39S) + automatic time (2S) - process standard C/T (41S) = 0S Time of man waiting for machine = automatic time (2S) - simultaneous working time of man and machine (0S)'
job_saturation = (
    f"Analyze the following text and determine whether the job saturation value mentioned is lower than 95%.\n\n"
    f"Text: \"{text_info}\"\n\n"
    f"If a job saturation value is explicitly mentioned, check if it is lower than 95%. If so, respond with 'YES'. "
    f"If it is 95% or higher, respond with 'NO'. If no job saturation value is mentioned, respond with 'NO INFORMATION'."
)
input_msg = ChatMessage.from_str(job_saturation)
output = llm.chat([input_msg])
answer = output.message.content

print(answer)

NO INFORMATION


In [25]:
# ItemsToBeImproved: List[str], Description,ImprovementParameters: List[str]) -> List[str]:
items_to_be_improved = {}
for index, item in enumerate(ItemsToBeImproved[:2]):
    items_to_be_improved[item] = {}
    for parameter in tqdm(ImprovementParameters):
        query = (
            f"For the problem '{item}', provide a detailed and concise value or description for the improvement parameter '{parameter}'.\n"
            f"If the parameter is not applicable or no information is available, respond explicitly with 'NA'.\n\n"
            f"Ensure your response is clear, contextually relevant, and avoids ambiguity."
            )
        # answer = str(query_engine.query(query))
        # answer = str(query_engine.query(query))
        items_to_be_improved[item]['Description'] = Description[index]
        items_to_be_improved[item][parameter] = '1'
items_to_be_improved        

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<?, ?it/s]


{'The saturation of the lower glass point Xiaoli Pill is 65%': {'Description': 'Low job saturation(higher than95%)',
  'Improvement direction': '1',
  'Person responsible': '1',
  'Expected start date': '1',
  'Actual start date': '1',
  'Expected completion date': '1',
  'Actual completion date': '1',
  'Improve immediately': '1',
  'Confirmation': '1',
  'appendix': '1'},
 'Bottom glass electrophoresis tank+UVFixed baking operation saturation79.2%': {'Description': 'Low job saturation(higher than99%)',
  'Improvement direction': '1',
  'Person responsible': '1',
  'Expected start date': '1',
  'Actual start date': '1',
  'Expected completion date': '1',
  'Actual completion date': '1',
  'Improve immediately': '1',
  'Confirmation': '1',
  'appendix': '1'}}

In [26]:
job_saturation = (
    f"Analyze the following text and determine whether the job saturation value mentioned is lower than 95%.\n\n"
    f"Text: \"{text_info}\"\n\n"
    f"If a job saturation value is explicitly mentioned, check if it is lower than 95%. If so, respond with 'YES'. "
    f"If it is 95% or higher, respond with 'NO'. If no job saturation value is mentioned, respond with 'NO INFORMATION'."
)
input_msg = ChatMessage.from_str(job_saturation)
output = llm.chat([input_msg])
answer = output.message.content

In [27]:
Description = ['Low job saturation(higher than95%)',
 'Low job saturation(lower than95%)',
 'Low job saturation(lower than95%)']

In [28]:
from tqdm import tqdm
def generate_answers(ItemsToBeImproved: List[str],Description: List[str], ImprovementParameters: List[str]) -> List[str]:
    items_to_be_improved = {}
    for index, item in enumerate(ItemsToBeImproved[:3]):
        items_to_be_improved[item] = {}
        for parameter in tqdm(ImprovementParameters):
            job_saturation = (
                f"Analyze the following text and determine whether the job saturation value mentioned is lower than 95%.\n\n"
                f"Text: \"{Description[index]}\"\n\n"
                f"If a job saturation value is explicitly mentioned, check if it is lower than 95%. If so, respond with 'YES'. "
                f"If it is 95% or higher, respond with 'NO'. If no job saturation value is mentioned, respond with 'NO INFORMATION'."
            )
            input_msg = ChatMessage.from_str(job_saturation)
            output = llm.chat([input_msg])
            answer = output.message.content
            print(Description[index], answer)
            if answer == 'YES':
                query = (
                    f"For the problem '{item}', provide a detailed and concise value or description for the improvement parameter '{parameter}'.\n"
                    f"If the parameter is not applicable or no information is available, respond explicitly with 'NA'.\n\n"
                    f"Ensure your response is clear, contextually relevant, and avoids ambiguity."
                    )
                answer = str(query_engine.query(query))
                items_to_be_improved[item]['Description'] = Description[index]
                items_to_be_improved[item][parameter] = answer
    return items_to_be_improved

In [ ]:
# ItemsToBeImproved = output_obj.ItemsToBeImproved
# ImprovementParameters = output_obj.TrueCause

In [29]:
answers = generate_answers(ItemsToBeImproved, Description, ImprovementParameters)

 11%|████████████████████▍                                                                                                                                                                   | 1/9 [00:01<00:11,  1.41s/it]

Low job saturation(higher than95%) NO


 22%|████████████████████████████████████████▉                                                                                                                                               | 2/9 [00:02<00:08,  1.24s/it]

Low job saturation(higher than95%) NO


 33%|█████████████████████████████████████████████████████████████▎                                                                                                                          | 3/9 [00:03<00:07,  1.29s/it]

...

 78%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                         | 7/9 [00:27<00:07,  3.81s/it]

Low job saturation(lower than95%) YES


 89%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                    | 8/9 [00:31<00:03,  3.82s/it]

Low job saturation(lower than95%) YES


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:33<00:00,  3.70s/it]


In [30]:
answers

{'The saturation of the lower glass point Xiaoli Pill is 65%': {},
 'Bottom glass electrophoresis tank+UVFixed baking operation saturation79.2%': {'Description': 'Low job saturation(lower than95%)',
  'Improvement direction': 'Judgment of value and reduce tasks without added value, Inspection of movement quality and human engineering hazards, Automated level inspection, Merge and rearrange new job elements.',
  'Person responsible': 'LiXX',
  'Expected start date': '10/8/24',
  'Actual start date': '10/8/24',
  'Expected completion date': '10/15/24',
  'Actual completion date': '10/15/24',
  'Improve immediately': 'yes',
  'Confirmation': 'LiXX',
  'appendix': 'LiXX'},
 'Xiaoliwan wax+Paste conductive foam+Lower glass glue frame dispensing operation saturation75%': {'Description': 'Low job saturation(lower than95%)',
  'Improvement direction': "The improvement direction for the problem 'Xiaoliwan wax+Paste conductive foam+Lower glass glue frame dispensing operation saturation75%' is as

In [31]:
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)
flat_data = []
for parameter, metrics in answers.items():
    flat_metrics = flatten_dict(metrics)
    flat_data.append(flat_metrics)

In [ ]:
# flat_metrics

In [ ]:
# flat_data

In [ ]:
# answers = json.loads("""
# {'The saturation of the lower glass point Xiaoli Pill is 65%': 'Judgment of value and reduce tasks without added value, Inspection of movement quality and human engineering hazards: Reduce the number of movements, work with both hands at the same time, shorten the distance of movements, and make movements easier; eliminate human engineering hazards, Automated level inspection: simple and automated import, Merge and rearrange new job elements.',
#  'Bottom glass electrophoresis tank+UVFixed baking operation saturation 79.2%': 'The improvement direction for Bottom glass electrophoresis tank+UVFixed baking operation saturation 79.2% is to reduce tasks without added value, inspect movement quality and human engineering hazards, conduct automated level inspection, and merge and rearrange new job elements.',
#  'Xiaoliwan wax+Paste conductive foam+Lower glass glue frame dispensing operation saturation75%': 'The improvement direction for Xiaoliwan wax+Paste conductive foam+Lower glass glue frame dispensing operation saturation75% is to reduce the number of movements, work with both hands at the same time, shorten the distance of movements, and make movements easier; eliminate human engineering hazards.'}
# """)

In [ ]:
flat_data

In [32]:
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Flatten the nested dictionary
flat_data = []
for parameter, metrics in answers.items():
    flat_metrics = flatten_dict(metrics)
    flat_metrics['Items to be improved'] = parameter 
    flat_data.append(flat_metrics)

# Get all unique keys to use as CSV headers
headers = set()
for item in flat_data:
    headers.update(item.keys())

# Sort headers to ensure 'Items to be improved' comes first
headers = sorted(headers)
headers.insert(0, headers.pop(headers.index('Items to be improved')))
headers.insert(1, headers.pop(headers.index('Description')))
headers.insert(2, headers.pop(headers.index('Improvement direction')))
headers.insert(3, headers.pop(headers.index('Person responsible')))
headers.insert(4, headers.pop(headers.index('Expected start date')))
headers.insert(5, headers.pop(headers.index('Actual start date')))
headers.insert(6, headers.pop(headers.index('Expected completion date')))
headers.insert(7, headers.pop(headers.index('Actual completion date')))
headers.insert(8, headers.pop(headers.index('Improve immediately')))
headers.insert(9, headers.pop(headers.index('Confirmation')))
headers.insert(10, headers.pop(headers.index('appendix')))

# Write to CSV
with open('Report_format_2_complete.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headers)
    writer.writeheader()
    for row in flat_data:
        writer.writerow(row)

In [33]:
import pandas as pd
from IPython.core.display import HTML
pd.set_option('display.max_colwidth', 10)
out_df = pd.read_csv("Report_format_2_complete.csv")
html = out_df.to_html()
HTML(html)

,Items to be improved,Description,Improvement direction,Person responsible,Expected start date,Actual start date,Expected completion date,Actual completion date,Improve immediately,Confirmation,appendix
0,The saturation of the lower glass point Xiaoli Pill is 65%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bottom glass electrophoresis tank+UVFixed baking operation saturation79.2%,Low job saturation(lower than95%),"Judgment of value and reduce tasks without added value, Inspection of movement quality and human engineering hazards, Automated level inspection, Merge and rearrange new job elements.",LiXX,10/8/24,10/8/24,10/15/24,10/15/24,yes,LiXX,LiXX
2,Xiaoliwan wax+Paste conductive foam+Lower glass glue frame dispensing operation saturation75%,Low job saturation(lower than95%),"The improvement direction for the problem 'Xiaoliwan wax+Paste conductive foam+Lower glass glue frame dispensing operation saturation75%' is as follows: 1. Judgment of value and reduce tasks without added value 2. Inspection of movement quality and human engineering hazards: Reduce the number of movements, work with both hands at the same time, shorten the distance of movements, and make movements easier; eliminate human engineering hazards 3. Automated level inspection: simple and automated import 4. Merge and rearrange new job elements.",LiXX,10/8/24,10/8/24,10/15/24,10/15/24,yes,The improvement parameter 'Confirmation' for the problem 'Xiaoliwan wax+Paste conductive foam+Lower glass glue frame dispensing operation saturation75%' is 'yes'.,LiXX


In [34]:
out_df.to_excel("Report_format_2_complete.xlsx", index=False)

In [ ]:
#Creating new duration column for plotting the Gantt Chart

In [35]:
import os
import pandas as pd
file_path = '../llama_parse_al/Report_format_2_complete.csv'
df = pd.read_csv(file_path)
df.head()

,Items to be improved,Description,Improvement direction,Person responsible,Expected start date,Actual start date,Expected completion date,Actual completion date,Improve immediately,Confirmation,appendix
0,The sa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bottom...,Low jo...,Judgme...,LiXX,10/8/24,10/8/24,10/15/24,10/15/24,yes,LiXX,LiXX
2,Xiaoli...,Low jo...,The im...,LiXX,10/8/24,10/8/24,10/15/24,10/15/24,yes,The im...,LiXX


In [36]:
df['duration'] = pd.to_datetime(df['Actual completion date']) - pd.to_datetime(df['Actual start date'])

C:\Users\Muhammad Al-Amin\AppData\Local\Temp\ipykernel_23624\4031733434.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['duration'] = pd.to_datetime(df['Actual completion date']) - pd.to_datetime(df['Actual start date'])
C:\Users\Muhammad Al-Amin\AppData\Local\Temp\ipykernel_23624\4031733434.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['duration'] = pd.to_datetime(df['Actual completion date']) - pd.to_datetime(df['Actual start date'])


In [37]:
df.head()

,Items to be improved,Description,Improvement direction,Person responsible,Expected start date,Actual start date,Expected completion date,Actual completion date,Improve immediately,Confirmation,appendix,duration
0,The sa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,Bottom...,Low jo...,Judgme...,LiXX,10/8/24,10/8/24,10/15/24,10/15/24,yes,LiXX,LiXX,7 days
2,Xiaoli...,Low jo...,The im...,LiXX,10/8/24,10/8/24,10/15/24,10/15/24,yes,The im...,LiXX,7 days


In [38]:
# Position a column to a specified place in pandas
columns = list(df.columns)
columns.remove('duration')
insert_at = columns.index('Actual completion date') + 1
columns.insert(insert_at, 'duration')
df = df[columns]

In [39]:
# Save the excel file after formatting is done
df.to_excel('Report_format_2_complete_input.xlsx', index=False)

In [40]:
df.head()

,Items to be improved,Description,Improvement direction,Person responsible,Expected start date,Actual start date,Expected completion date,Actual completion date,duration,Improve immediately,Confirmation,appendix
0,The sa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
1,Bottom...,Low jo...,Judgme...,LiXX,10/8/24,10/8/24,10/15/24,10/15/24,7 days,yes,LiXX,LiXX
2,Xiaoli...,Low jo...,The im...,LiXX,10/8/24,10/8/24,10/15/24,10/15/24,7 days,yes,The im...,LiXX


In [41]:
import os
file_path = '../llama_parse_al/Report_format_2_complete_input.xlsx'
os.path.exists(file_path)

True

In [42]:
from openpyxl import load_workbook
from openpyxl.chart import BarChart, Reference
from openpyxl.chart.label import DataLabelList
# Step 1: Load the Excel file
# file_path = "data.xlsx"  # Replace with your Excel file path
workbook = load_workbook(file_path)
sheet = workbook.active  # Use the active sheet (or specify by name: workbook['SheetName'])

# Step 2: Identify the data range (e.g., A1:B5)
# Assuming the first column has categories (e.g., tasks) and the second has values (e.g., counts)
categories = Reference(sheet, min_col=1, min_row=2, max_row=sheet.max_row)  # Task names
values = Reference(sheet, min_col=9, min_row=1, max_row=sheet.max_row)  # Values including header

# Step 3: Create a horizontal bar chart
chart = BarChart()
chart.type = "bar"  # Horizontal bar chart
chart.title = "Timeline Chart"
chart.y_axis.title = "Days"
chart.x_axis.title = "Tasks"
# Add data and categories to the chart
chart.add_data(values, titles_from_data=True)
chart.set_categories(categories)
# Step 5: Add data labels
data_labels = DataLabelList()  # Create a DataLabelList object
data_labels.showVal = True  # Show values on the bars
data_labels.position = 'inBase'
chart.dLbls = data_labels
chart.legend = None

# Step 4: Add the chart to the worksheet
sheet.add_chart(chart, "M2")  # Place the chart in column D, row 2

# Step 5: Save the updated Excel file
workbook.save("updated_data.xlsx")
print("Chart added to 'updated_data.xlsx'")

Chart added to 'updated_data.xlsx'


In [ ]:
chart.dLbls

In [43]:
from openpyxl import load_workbook
workbook = load_workbook("Report_format_2_complete.xlsx")

In [44]:
# Select the active worksheet (or specify the sheet by name)
sheet = workbook.active  # or workbook["SheetName"]

# Insert a new row at the beginning
sheet.insert_rows(1)  # Insert an empty row at the top

# Merge two cells (e.g., A1 and B1)
sheet.merge_cells("E1:F1")

# Write data into the merged cells
sheet["E1"] = "Date"

# Save the workbook
workbook.save("example.xlsx")

print("New row added at the beginning!")


New row added at the beginning!


In [45]:
# Try improving the graph to make an interactive Gantt Chart
## Final version of the graph of Gantt Chart

import pandas as pd
import plotly.express as px
from openpyxl import load_workbook
from openpyxl.drawing.image import Image

# Step 1: Read Excel file
file_path = './data/tasks.xlsx'
df = pd.read_excel(file_path)

# Step 2: Ensure columns are in the correct format
df['Start'] = pd.to_datetime(df['Start'])
df['Finish'] = pd.to_datetime(df['Finish'])

# Step 3: Add a column for data labels (e.g., Task names or durations)
df['Label'] =  (df['Finish'] - df['Start']).dt.days.astype(str)
# df['Label'] = df['Task'] + " (" + ((df['Finish'] - df['Start']).dt.days.astype(str)) + " days)"

# Step 4: Create a Gantt chart with data labels
fig = px.timeline(
    df,
    x_start="Start",
    x_end="Finish",
    y="Task",
    color="Resource",
    text="Label",  # Overlay Task names and durations
    title="Gantt Chart with Custom X-Axis"
)

# Step 5: Customize the x-axis
fig.update_traces(textposition="auto", textfont=dict(size=12, color="white"))

fig.update_yaxes(categoryorder="trace")  # Optional: Sort tasks by total ascending
fig.update_layout(
    xaxis_title="Project Timeline (Dates)",  # Custom x-axis label
    title_x=0.5  # Center the chart title
)

# Step 6: Save the chart as an image
image_file = "gantt_chart_with_custom_xaxis.png"
fig.write_image(image_file)  # Requires kaleido library (install using `pip install kaleido`)

# Step 7: Load the Excel file
wb = load_workbook(file_path)
ws = wb.active  # Use the first sheet (or specify a sheet name)

# Step 8: Insert the image into the Excel file
img = Image(image_file)
img.anchor = "E2"  # Position to embed the image (e.g., cell E2)
ws.add_image(img)

# Step 9: Save the updated Excel file
output_file = "tasks_with_custom_xaxis.xlsx"
wb.save(output_file)

print(f"Gantt chart with custom x-axis embedded in {output_file}")

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido
